In [1]:
import warnings 
warnings.filterwarnings('ignore')
from tqdm.notebook import tqdm

In [2]:
import pandas as pd
import numpy as np
import random

import pyLDAvis
import pyLDAvis.gensim_models

In [3]:
# Import required libraries
import pandas as pd
import re
import nltk
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import CountVectorizer
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score, classification_report


nltk.download('all')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize
from nltk.sentiment import SentimentIntensityAnalyzer


# Download NLTK resources
nltk.download('stopwords')
nltk.download('punkt')



[nltk_data] Downloading collection 'all'
[nltk_data]    | 
[nltk_data]    | Downloading package abc to
[nltk_data]    |     C:\Users\farah\AppData\Roaming\nltk_data...
[nltk_data]    |   Package abc is already up-to-date!
[nltk_data]    | Downloading package alpino to
[nltk_data]    |     C:\Users\farah\AppData\Roaming\nltk_data...
[nltk_data]    |   Package alpino is already up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger to
[nltk_data]    |     C:\Users\farah\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger is already up-
[nltk_data]    |       to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_eng to
[nltk_data]    |     C:\Users\farah\AppData\Roaming\nltk_data...
[nltk_data]    |   Package averaged_perceptron_tagger_eng is already
[nltk_data]    |       up-to-date!
[nltk_data]    | Downloading package averaged_perceptron_tagger_ru to
[nltk_data]    |     C:\Users\farah\AppData\Roaming\nltk_data...
[

True

In [4]:
df_car = pd.read_csv('car_5_brands.csv')
df_car

,Unnamed: 0,Rating,car_year,brand_name,date,review
0,0,5.00,2018,Audi,2018-07-11,BEST ALL AROUND PURPOSE CROSSOVER SUV I have n...
1,1,5.00,2018,Audi,2018-06-24,Best car This is a wonderful car. The technol...
2,2,5.00,2018,Audi,2018-05-02,Great Buy Do your home work
3,3,5.00,2018,Audi,2017-12-07,Fun Car Great ride. Loaded with technology. St...
4,4,5.00,2018,Audi,2017-10-25,Best luxury SUV w/ perfect comfort/sport balan...
...,...,...,...,...,...,...
31933,31933,4.75,2015,Mercedes-Benz,2015-06-04,One week in but I love it. I just got the car...
31934,31934,4.00,2015,Mercedes-Benz,2015-05-19,A delightful car I have been driving the GLA25...
31935,31935,3.00,2015,Mercedes-Benz,2015-02-21,This car is amazing but visibility is poor We ...
31936,31936,2.75,2015,Mercedes-Benz,2015-01-20,Steering Wheel Module When I first bought this...


## data info

In [ ]:
#data = df_car.drop(columns=['Unnamed: 0'])

In [13]:
df_car.brand_name.value_counts()
df_car.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 31938 entries, 0 to 31937
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Rating      31938 non-null  float64
 1   car_year    31938 non-null  int64  
 2   brand_name  31938 non-null  object 
 3   date        31938 non-null  object 
 4   review      31938 non-null  object 
dtypes: float64(1), int64(1), object(3)
memory usage: 1.2+ MB


## pre-processing

In [14]:
stop_words = list(stopwords.words('english'))

stop_words.extend(['porsche,' 'mercede','comfortsport', 'mercedes','mercedes-benz', 'honda','toyota','audi', 'benz','bentley','lexus',
                  'nissan','volvo','drive','nt','like','vehicle','infiniti','good','miles','corvette','come','edmund','lotus','diego','snake',
                 'porsche', 'cayman','bought','year','minute','chicago','car','home', 'work','think','suv','people','edmunds',
                  'cabriolet','lexuss','japan','husband','baby','range', 'rover','cadillac','cadillacs','michelin','texas','second',
                   'awsome','one','now', 'take', 'give', 'new','levinson','road','love','sedan','wife','sport','bang','tank',
                   'truck','lemon','imho','pathfinder','infinity','convertible','allroad','conv','bike','ski','grocery','mclass'
                  ,'hardtop','club','hubby','child','zoom','test','etc','brain','ashamed','carmax','alpina','rocketship','great','germany',
                  'autobahn','mercedez','bmw'])

In [15]:
df_car.review

0        BEST ALL AROUND PURPOSE CROSSOVER SUV I have n...
1        Best car This is a wonderful car.  The technol...
2                              Great Buy Do your home work
3        Fun Car Great ride. Loaded with technology. St...
4        Best luxury SUV w/ perfect comfort/sport balan...
                               ...                        
31933    One week in but I love it.  I just got the car...
31934    A delightful car I have been driving the GLA25...
31935    This car is amazing but visibility is poor We ...
31936    Steering Wheel Module When I first bought this...
31937    Fun crossover/hatchback but not an SUV. I've h...
Name: review, Length: 31938, dtype: object

In [16]:
from nltk.stem import WordNetLemmatizer

lemmatizer = WordNetLemmatizer()

In [17]:
def tokenisation_pos_stopword_lemmatize(x):
  ''' This function was created to tokenise,POS tagging, Remove Stopwords & then lemmatise the Reviews. '''
  tokens = nltk.word_tokenize(x)
  #print (nltk.pos_tag(tokens))
  #POS-tagging
  tags = nltk.pos_tag(tokens)
  pos_tags_words = [t for t in tags if t[1] in["JJ","JJR","JJS","NN","NNP","NNS","NNPS","VB", "VBD" ,"VBG" ,"VBN" ,"VBP", "VBZ"] ]
  #stop-word removal
  filtered_words = [t[0] for t in pos_tags_words]
  filtered_words2 = [w for w in filtered_words if not w.lower() in stop_words]
  #lemmatization with lowercase function
  lemmatized_output = [lemmatizer.lemmatize(w).lower() for w in filtered_words2]
  return lemmatized_output

In [18]:
#review cleaned column after tokenize, remove stopword and lemmatize
df_car['review_cleaned'] = df_car['review'].apply(tokenisation_pos_stopword_lemmatize)


In [19]:
df_car = df_car.reset_index()
df_car.drop('index',axis=1,inplace=True)
#df_car.to_csv('data_sentiment_pp.csv')


In [20]:
df_car

,Rating,car_year,brand_name,date,review,review_cleaned
0,5.00,2018,Audi,2018-07-11,BEST ALL AROUND PURPOSE CROSSOVER SUV I have n...,"[around, purpose, crossover, seen, driven, 've..."
1,5.00,2018,Audi,2018-06-24,Best car This is a wonderful car. The technol...,"[best, wonderful, technology, adaptable, comfo..."
2,5.00,2018,Audi,2018-05-02,Great Buy Do your home work,[buy]
3,5.00,2018,Audi,2017-12-07,Fun Car Great ride. Loaded with technology. St...,"[fun, ride, loaded, technology, steering, tigh..."
4,5.00,2018,Audi,2017-10-25,Best luxury SUV w/ perfect comfort/sport balan...,"[best, luxury, w/, perfect, comfort/sport, bal..."
...,...,...,...,...,...,...
31933,4.75,2015,Mercedes-Benz,2015-06-04,One week in but I love it. I just got the car...,"[week, got, procrastinating, couple, week, loo..."
31934,4.00,2015,Mercedes-Benz,2015-05-19,A delightful car I have been driving the GLA25...,"[delightful, driving, gla250, half, problem, p..."
31935,3.00,2015,Mercedes-Benz,2015-02-21,This car is amazing but visibility is poor We ...,"[amazing, visibility, poor, rear, window, chan..."
31936,2.75,2015,Mercedes-Benz,2015-01-20,Steering Wheel Module When I first bought this...,"[steering, wheel, module, first, month, drivin..."


In [21]:
# Save the updated dataset
df_car.to_csv("data_sent_pp.csv", index=False)

print("Balanced dataset saved as 'data_sent_pp.csv'")

Balanced dataset saved as 'data_sent_pp.csv'


## aspect based sentiments

In [34]:
df_car = pd.read_csv('data_pp_lda.csv')
df_car

,Rating,car_year,brand_name,date,review,review_cleaned,Dominant_Topic,Aspect
0,5.00,2018,Audi,11/7/2018,BEST ALL AROUND PURPOSE CROSSOVER SUV I have n...,"['around', 'purpose', 'crossover', 'seen', 'dr...",3,performance
1,5.00,2018,Audi,24/6/2018,Best car This is a wonderful car. The technol...,"['best', 'wonderful', 'technology', 'adaptable...",3,performance
2,5.00,2018,Audi,2/5/2018,Great Buy Do your home work,['buy'],2,price
3,5.00,2018,Audi,7/12/2017,Fun Car Great ride. Loaded with technology. St...,"['fun', 'ride', 'loaded', 'technology', 'steer...",3,performance
4,5.00,2018,Audi,25/10/2017,Best luxury SUV w/ perfect comfort/sport balan...,"['best', 'luxury', 'w/', 'perfect', 'comfort/s...",3,performance
...,...,...,...,...,...,...,...,...
31933,4.75,2015,Mercedes-Benz,4/6/2015,One week in but I love it. I just got the car...,"['week', 'got', 'procrastinating', 'couple', '...",4,fuel efficiency
31934,4.00,2015,Mercedes-Benz,19/5/2015,A delightful car I have been driving the GLA25...,"['delightful', 'driving', 'gla250', 'half', 'p...",5,quality
31935,3.00,2015,Mercedes-Benz,21/2/2015,This car is amazing but visibility is poor We ...,"['amazing', 'visibility', 'poor', 'rear', 'win...",4,fuel efficiency
31936,2.75,2015,Mercedes-Benz,20/1/2015,Steering Wheel Module When I first bought this...,"['steering', 'wheel', 'module', 'first', 'mont...",5,quality


In [35]:
sia = SentimentIntensityAnalyzer()

def sentimenter(x) :
  result = sia.polarity_scores(x)
  return result

In [36]:
''' we have classified sentiments based on compound score & we have decided tweak the limits a bit. '''
df_car['sentiment score'] = df_car['review'].apply(sentimenter)
def sent_score_string(x):
  if (x['compound']>= 0.05):
    return 'positive'
  elif (x['compound']> -0.05 and x['compound']< 0.05) :
    return 'neutral'
  elif (x['compound'] <= -0.05):
    return 'negative'

df_car['relative sentiments'] = df_car['sentiment score'].apply(sent_score_string)

In [37]:
# Count occurrences of each sentiment
sentiment_counts = df_car['relative sentiments'].value_counts()

# Print the result
print(sentiment_counts)


relative sentiments
positive    26578
negative     4465
neutral       895
Name: count, dtype: int64


In [38]:
output_path = 'data_pp_lda_sentiment.csv'
df_car.to_csv(output_path, index=False)
print(f"File saved successfully at {output_path}")


File saved successfully at data_pp_lda_sentiment.csv


## sampling (SMOTE)

In [ ]:
!pip install imbalanced-learn

In [42]:
import sklearn
import imblearn

print(sklearn.__version__)  # Check scikit-learn version
print(imblearn.__version__)  # Check imbalanced-learn version


1.6.1
0.13.0


In [3]:
import pandas as pd
from imblearn.combine import SMOTETomek
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.metrics import classification_report, confusion_matrix, accuracy_score

# Load the dataset
file_path = 'data_pp_lda_sentiment.csv'  # Replace with your actual file path
df = pd.read_csv(file_path)

# Define features (X) and target variable (y)
X = df[['Rating', 'car_year']]  # Adjust features as per your dataset
y = df['relative sentiments']  # Target variable: positive, neutral, negative

# Apply SMOTE + Tomek Links
smote_tomek = SMOTETomek(random_state=42)
X_resampled, y_resampled = smote_tomek.fit_resample(X, y)

# Combine resampled features and target into a new DataFrame
resampled_data = pd.DataFrame(X_resampled, columns=['Rating', 'car_year'])
resampled_data['relative sentiments'] = y_resampled

# Save the resampled data to a new CSV file
output_file_path = 'balanced_data.csv'
resampled_data.to_csv(output_file_path, index=False)

output_file_path


'balanced_data.csv'

In [2]:
import pandas as pd
from sklearn.utils import shuffle

# Load your dataset
file_path = "data_sentiment_pp.csv"
df = pd.read_csv(file_path)

# Define the desired counts for each sentiment
desired_counts = {
    "positive": 26491,
    "negative": 26176,
    "neutral": 26123
}

# Create a new DataFrame to store the additional rows
new_data = pd.DataFrame()

# Start index for new rows
new_index_start = df.index.max() + 1

# Balance each sentiment category
for sentiment, target_count in desired_counts.items():
    # Filter rows with the current sentiment
    sentiment_df = df[df["relative sentiments"] == sentiment]
    
    # Calculate the number of rows to add
    current_count = len(sentiment_df)
    rows_to_add = target_count - current_count
    
    # Duplicate rows if needed
    if rows_to_add > 0:
        # Sample rows with replacement to match the desired count
        additional_rows = sentiment_df.sample(n=rows_to_add, replace=True, random_state=42)
        # Assign new indices to the additional rows
        additional_rows.index = range(new_index_start, new_index_start + rows_to_add)
        new_index_start += rows_to_add
        
        # Add the additional rows to the new_data DataFrame
        new_data = pd.concat([new_data, additional_rows])

# Add the new data to the original dataset
df = pd.concat([df, new_data])

# Save the updated dataset
df.to_csv("balanced_data_smote.csv", index=False)

print("Balanced dataset saved as 'balanced_data_smote.csv'")


FileNotFoundError: [Errno 2] No such file or directory: 'data_sentiment_pp.csv'